![](https://raw.githubusercontent.com/MomsFriendlyRobotCompany/yivo/main/docs/yivo.png)

# Yivo

Kevin Walchko

---

This is a derivation off of MSP which seems to be `int`s rather than supporting `float`s for numbers.

I am going to use some of the ideas of MSP, but I won't really implement it.

| Header | Data | Checksum |
|--------|------|----------|
| `$M[>,<,!]NT` | `uint8_t[N]`|`uint8_t`|

| 0 | 1 | 2 | 3 | 4 | ... | -1 |
|---|---|---|---|---|-----|----|
|`$`|`M`| D | N | T | ... | checksum |

| Use    |Header|data length|code|data|checksum|
|--------|------|-----------|----|----|--------|
|Request Output | `$M>`| N | T | data | chksum |
|Response Input | `$M<`| N | T | data | chksum |
|Error   | `$M!`| 0 | error code | `None` | chksum |
|Format  |`uint8_t[3]`|`uint8_t`|`uint8_t`|`uint8_t[N]`|`uint8_t`|

data length can be 0 in case of no param command

# References

- github: [yivo](https://github.com/MomsFriendlyRobotCompany/yivo)
- [MSPv2](https://github.com/iNavFlight/inav/wiki/MSP-V2)
- github: [MSP(C++)](https://github.com/christianrauch/msp)
- [Python `struct` format](https://docs.python.org/3/library/struct.html#format-characters)
- [MSP-Python3](https://github.com/LukeARohl/MSP-Python3)
- github: [Betaflight MSP](https://github.com/betaflight/betaflight/tree/master/src/main/msp)
- github: [MultiWii_2_4](https://github.com/xdu-aero-association/MultiWii_2_4)

In [7]:
# reload library
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from colorama import Fore
from yivo import Yivo, MsgIDs, checksum

rr = Yivo()
msg = rr.pack(MsgIDs.RAW_IMU, range(9))
print("TX:",msg)

data = rr.unpack(msg)
print("RX:",data)

TX: b'$M>$f\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@\x00\x00\x00A\xfc'
RX: IMU_RAW(ax=0.0, ay=1.0, az=2.0, gx=3.0, gy=4.0, gz=5.0, mx=6.0, my=7.0, mz=8.0)


In [3]:
msg = rr.pack(MsgIDs.IMU_AGMQT, range(15))
print("TX:",msg)

data = rr.unpack(msg)
print("RX:",data)

TX: b'$M><\x8c\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@\x00\x00\x00A\x00\x00\x10A\x00\x00 A\x00\x000A\x00\x00@A\x00\x00PA\x0e\x00\x00\x00Q'
RX: ImuAGMQT(ax=0.0, ay=1.0, az=2.0, wx=3.0, wy=4.0, wz=5.0, mx=6.0, my=7.0, mz=8.0, qw=9.0, qx=10.0, qy=11.0, qz=12.0, temperature=13.0, ts=14)


In [4]:
for msgid in MsgIDs:
    print(msgid)

MsgIDs.CALIBRATION_DATA
MsgIDs.SET_CALIBRATION_DATA
MsgIDs.REBOOT
MsgIDs.IDENT
MsgIDs.STATUS
MsgIDs.RAW_IMU
MsgIDs.SERVO
MsgIDs.MOTOR
MsgIDs.RC
MsgIDs.RAW_GPS
MsgIDs.COMP_GPS
MsgIDs.ATTITUDE
MsgIDs.ALTITUDE
MsgIDs.ANALOG
MsgIDs.RC_TUNING
MsgIDs.PID
MsgIDs.MOTOR_PINS
MsgIDs.WP
MsgIDs.SERVO_CONF
MsgIDs.IMU_AGMQT
MsgIDs.IMU_AGMT
MsgIDs.IMU_AGT
MsgIDs.MAGNETIC
MsgIDs.RANGE
MsgIDs.TEMP_PRES
MsgIDs.SET_RAW_RC
MsgIDs.SET_RAW_GPS
MsgIDs.SET_PID
MsgIDs.SET_BOX
MsgIDs.SET_RC_TUNING
MsgIDs.ACC_CALIBRATION
MsgIDs.MAG_CALIBRATION
MsgIDs.SET_WP
MsgIDs.SET_HEAD
MsgIDs.SET_MOTOR


In [26]:
def test_yivo():
    rr = Yivo()
    for k,v in list(Yivo.msgInfo.items()):
        print(f"[ {Fore.GREEN}{k}{Fore.RESET} ]--------------------------------")
        fmt, obj = v
        size = fmt.size - 6
        testsz = len(obj._fields)
        msg = rr.pack(k, range( testsz ))
        print(f"  TX: {Fore.CYAN}{msg}{Fore.RESET}")
        # print(f"  {len(msg[5:-1])} ?? {size}")
        # print("  cs again", checksum(size,k,msg[5:-1]), f"?? {msg[-1]}")
        # print(f"  {k} ?? {msg[4]}")

        data = rr.unpack(msg)
        print(f"  RX: {data}")
        assert tuple(data) == tuple([float(x) for x in range(testsz)])
        
test_yivo()

[ 100 ]--------------------------------
  TX: b'$M>\x07d\x00\x01\x02\x03\x00\x00\x00c'
  RX: IDENT(version=0, multitype=1, msp_version=2, capability=3)
[ 102 ]--------------------------------
  TX: b'$M>\x12f\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00|'
  RX: IMU_RAW(ax=0, ay=1, az=2, gx=3, gy=4, gz=5, mx=6, my=7, mz=8)
[ 140 ]--------------------------------
  TX: b'$M><\x8c\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@\x00\x00\x00A\x00\x00\x10A\x00\x00 A\x00\x000A\x00\x00@A\x00\x00PA\x0e\x00\x00\x00Q'
  RX: ImuAGMQT(ax=0.0, ay=1.0, az=2.0, wx=3.0, wy=4.0, wz=5.0, mx=6.0, my=7.0, mz=8.0, qw=9.0, qx=10.0, qy=11.0, qz=12.0, temperature=13.0, ts=14)
[ 141 ]--------------------------------
  TX: b'$M>,\x8d\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@\x00\x00\x00A\x00\x00\x10A\n\x00\x00\x00D'
  RX: ImuAGMT(ax=0.0, ay=1.0, az=2.0, wx=3.0, wy=4.0, 